In [ ]:
import smtplib
import pyodbc

conn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=ETLDB;UID=sa;PWD=your_password')
cursor = conn.cursor()

cursor.execute("""
    SELECT JobName, ErrorMessage, StartTime FROM ETLJobStatus
    WHERE Status = 'Failed' AND StartTime >= DATEADD(HOUR, -24, GETDATE())
""")
failed_jobs = cursor.fetchall()

if failed_jobs:
    message = "\n".join([f"{row.JobName} failed at {row.StartTime} with error: {row.ErrorMessage}" for row in failed_jobs])

    server = smtplib.SMTP('smtp.yourmailserver.com', 587)
    server.starttls()
    server.login('your_email@example.com', 'your_password')
    server.sendmail('your_email@example.com', 'admin@example.com', f"Subject: ETL Job Failure Alert\n\n{message}")
    server.quit()
